In [1]:
from IPython.display import display

In [2]:
from pdstools.explanations import GradientBoostGlobalExplanations, ExplanationsExplorer, ExplanationsDataLoader

import datetime
import polars as pl

## Aggregate data exported from infinity

If this step has not already been done, you need to aggregate the explanation data exported from infinity. Unless otherwise specified, the resulting aggrgeated data will be stored in the `.tmp/aggregate/data` directory.

This step can be skipped if you have already aggregated the data.

In [3]:
explanations = GradientBoostGlobalExplanations(
    data_folder='../../data/explanations/',
    model_name='TelcoGb_CustomModel',
    to_date=datetime.date(2025,6,30)
)
explanations.generate_aggregations(overwrite=True)

## Simple Data Exploration
For quick overview of the data, you can use the ExplanationsExplorer class. It provides a simple interface to select a context and then plot the corrsponding predictor contribution plots.

In [4]:
explorer = ExplanationsExplorer()

### Selecting a context
Select the desired context from the list available on the right.
* Selecting 'Any' means the plots will display and aggegation across all contexts.
* Selecting a specific context will result in plots which aggregate the data for that context only.

If you have a very large list of possible contexts, you can filter the list on the right by selecting specific context keys from the comboboxes on the left.

In [5]:
explorer.display_context_selector()

GridBox(children=(HTML(value='<h3>Select Context Filters</>', layout=Layout(width='auto')), HTML(value='<h3>Co…

## Plotting the explanation data

The method shown below can be used to plot the explanation data for context selected above.
* The first plot shows the top 10 predictors sorted by their average contributions to predictions, the number can be changed by passing a different value to the `top_n` parameter.
* The subsequent plots show the avergae prediction contributions for the different values of top predictors.
    * Numeric predictor values are binned into deciles.
    * Categorical predictors are sorted and limited to the top 10 highest contributing values.

Note that the number of top predictors and top values can be changed by passing different values for the `top_n` and `top_k` parameters, respectively.

Additionally, if you are interested seeing the least contribution predictors, the `descending` parameter can be set to `False`. This will plot the least contributing predictors instead of the most contributing ones.

In [6]:
plots = explorer.plot_contributions(top_n = 5, top_k = 5)

No context selected, plotting overall contributions.


## Advaced Data Exploration
For more advanced data explortation you can directly use the ExplanationsDataLoader and ExplanationsDataLoader classes. These classes provide more flexibility in how the data is loaded and processed. Allowing you to inspect the data before plotting.

In [7]:
loader = ExplanationsDataLoader(data_location='.tmp/aggregated_data')

In [8]:
top_predictors = loader.get_top_n_predictor_contribution_overall(top_n = 5, remaining=False)
top_predictors

partition,predictor_name,predictor_type,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max
str,str,str,f64,f64,f64,f64,i64,f64,f64
"""whole_model""","""IH.PegaBatch.E2E Test.Decline.…","""NUMERIC""",-0.007047,0.007726,-0.00001,0.00001,48502,-0.009215,0.010256
"""whole_model""","""pyName""","""SYMBOLIC""",-0.004863,0.005087,-1.3287e-7,1.3898e-7,50000,-0.015087,0.007153
"""whole_model""","""IH.PegaBatch.E2E Test.Decline.…","""SYMBOLIC""",-0.001276,0.001277,-0.000003,0.000003,48502,-0.002626,0.000895
"""whole_model""","""USG_7""","""NUMERIC""",-0.000687,0.000693,-1.8313e-7,1.8461e-7,50000,-0.001835,0.000279
"""whole_model""","""CUS_3""","""SYMBOLIC""",-0.000589,0.000597,-4.4016e-9,4.4395e-9,50000,-0.000867,0.000276


We can get the top predictors and inspect their most influential values

In [9]:
predictors = top_predictors.select(pl.col('predictor_name')).unique().to_series().to_list()
loader.get_top_k_predictor_value_contribution_overall(predictors=predictors, top_k = 5, remaining=False)

partition,predictor_name,predictor_type,bin_order,bin_contents,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max,sort_column,sort_value
str,str,str,i64,str,f64,f64,f64,f64,i64,f64,f64,str,f64
"""whole_model""","""CUS_3""","""SYMBOLIC""",185,"""V434""",-0.000783,0.000783,-4.6954e-8,4.6954e-8,3,-0.000785,-0.000779,"""contribution""",0.000783
"""whole_model""","""CUS_3""","""SYMBOLIC""",217,"""V86""",-0.000784,0.000784,-4.7063e-8,4.7063e-8,3,-0.000786,-0.000784,"""contribution""",0.000784
"""whole_model""","""CUS_3""","""SYMBOLIC""",189,"""V183""",-0.000812,0.000812,-4.8735e-8,4.8735e-8,3,-0.000834,-0.000768,"""contribution""",0.000812
"""whole_model""","""CUS_3""","""SYMBOLIC""",170,"""VNA""",-0.000815,0.000815,-4.8911e-8,4.8911e-8,3,-0.000839,-0.000768,"""contribution""",0.000815
"""whole_model""","""CUS_3""","""SYMBOLIC""",211,"""V387""",-0.000835,0.000835,-5.0086e-8,5.0086e-8,3,-0.000835,-0.000835,"""contribution""",0.000835
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""whole_model""","""pyName""","""SYMBOLIC""",69,"""P60""",-0.007748,0.007748,-0.000084,0.000084,539,-0.010173,-0.000401,"""contribution""",0.007748
"""whole_model""","""pyName""","""SYMBOLIC""",6,"""P50""",-0.0094,0.0094,-0.000111,0.000111,591,-0.012273,-0.001095,"""contribution""",0.0094
"""whole_model""","""pyName""","""SYMBOLIC""",59,"""P63""",-0.01007,0.01007,-0.00011,0.00011,544,-0.012526,-0.005325,"""contribution""",0.01007


Let's repeat the same again, but this time we will inspect a specific context, instead of the entire model.

In [10]:
import random
context_info = random.choice(loader.get_unique_contexts_list())
print('Selected random context: \n')
for key, value in context_info.items():
    print(f'{key}: {value}')
top_predictors_for_selected_context = loader.get_top_n_predictor_contribution_by_context(context=context_info, top_n=5, remaining=False)
top_predictors_for_selected_context


Selected random context: 

pyChannel: pyChannel_0
pyDirection: pyDirection_1
pyGroup: pyGroup_1
pyIssue: pyIssue_1
pyName: pyName_2


partition,predictor_name,predictor_type,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max
str,str,str,f64,f64,f64,f64,i64,f64,f64
"""{""partition"": {""pyChannel"": ""p…","""IH.PegaBatch.E2E Test.Decline.…","""NUMERIC""",-0.006817,0.007564,-0.000009,0.00001,519,-0.009085,0.009024
"""{""partition"": {""pyChannel"": ""p…","""pyName""","""SYMBOLIC""",-0.004672,0.005055,-1.3255e-7,1.4001e-7,539,-0.014134,0.006057
"""{""partition"": {""pyChannel"": ""p…","""IH.PegaBatch.E2E Test.Decline.…","""SYMBOLIC""",-0.001273,0.001276,-0.000003,0.000003,519,-0.002626,0.000537
"""{""partition"": {""pyChannel"": ""p…","""USG_7""","""NUMERIC""",-0.000687,0.000696,-1.8323e-7,1.8540e-7,539,-0.001676,0.000238
"""{""partition"": {""pyChannel"": ""p…","""CUS_3""","""SYMBOLIC""",-0.00056,0.000568,-1.5315e-8,1.5488e-8,539,-0.000839,0.000196


In [11]:
predictors_for_selected_context = top_predictors_for_selected_context.select(pl.col('predictor_name')).unique().to_series().to_list()
loader.get_top_k_predictor_value_contribution_by_context(predictors=predictors_for_selected_context, top_k=5, context=context_info, remaining=False)

partition,predictor_name,predictor_type,bin_order,bin_contents,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max,sort_column,sort_value
str,str,str,i64,str,f64,f64,f64,f64,i64,f64,f64,str,f64
"""{""partition"": {""pyChannel"": ""p…","""CUS_3""","""SYMBOLIC""",43,"""V300""",-0.000767,0.000767,-0.000001,0.000001,1,-0.000767,-0.000767,"""contribution""",0.000767
"""{""partition"": {""pyChannel"": ""p…","""CUS_3""","""SYMBOLIC""",53,"""V254""",-0.000768,0.000768,-0.000001,0.000001,1,-0.000768,-0.000768,"""contribution""",0.000768
"""{""partition"": {""pyChannel"": ""p…","""CUS_3""","""SYMBOLIC""",44,"""V313""",-0.000771,0.000771,-0.000001,0.000001,1,-0.000771,-0.000771,"""contribution""",0.000771
"""{""partition"": {""pyChannel"": ""p…","""CUS_3""","""SYMBOLIC""",64,"""V426""",-0.000782,0.000782,-0.000001,0.000001,1,-0.000782,-0.000782,"""contribution""",0.000782
"""{""partition"": {""pyChannel"": ""p…","""CUS_3""","""SYMBOLIC""",59,"""V9""",-0.000838,0.000838,-0.000002,0.000002,1,-0.000838,-0.000838,"""contribution""",0.000838
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{""partition"": {""pyChannel"": ""p…","""pyName""","""SYMBOLIC""",61,"""P60""",-0.008078,0.008078,-0.000075,0.000075,5,-0.008913,-0.006804,"""contribution""",0.008078
"""{""partition"": {""pyChannel"": ""p…","""pyName""","""SYMBOLIC""",86,"""P63""",-0.009192,0.009192,-0.000034,0.000034,2,-0.010552,-0.007833,"""contribution""",0.009192
"""{""partition"": {""pyChannel"": ""p…","""pyName""","""SYMBOLIC""",12,"""P50""",-0.009487,0.009487,-0.000158,0.000158,9,-0.01127,-0.008631,"""contribution""",0.009487
